In [1]:
import numpy as np
import mne
import matplotlib.pyplot as plt
import os
from numpy import diff
import statistics 

In [2]:
#Takes period segment, voltage threshold, min spike duration in seconds, max spike duration in seconds, and the sampling frequency, returns the amount of spikes in the segment udner those conditions.

def spikeamount(segment, spike_thresh, spike_min, spike_max, samp_freq):
    sample_count = 0
    spikes = 0
    spike_min = spike_min*samp_freq
    spike_max = spike_max*samp_freq
    time_of_spike = []
    #Mean_value = statistics.mean(segment)
    
    for sample in segment:
        if sample > spike_thresh:
            sample_count += 1
            
        elif sample_count > spike_min and sample_count < spike_max:
            spikes += 1
            sample_count = 0
        else:
            sample_count = 0
    return spikes


#Takes a period as list of samples, desired segment length in seconds and the sampling frequency, returns a list of segments of the period {{},{},...{}}
def splitperiod(period, segment_length, sampling_freq):
    segments = []
    current_segment = []
    sample_seglen = segment_length * sampling_freq
    for i, sample in enumerate(period):
        current_segment.append(sample)
        if i%sample_seglen == 0 and i!=0:
            segments.append(current_segment)
            current_segment = []
    return segments

#takes list of spike amounts, returns list of smoothed spike amounts (smoothed spike amount is the average of M neighbour spike amount)
def smoothspikes(spikes, M):
    smoothspikes = []
    window_min = -(M//2)
    window_max = M//2
    for spike in spikes:
        curravg = 0
        if window_min<0:
            for i in range(0,M):
                curravg+=spikes[i]
            smoothspikes.append(curravg/M)
        elif window_max>=len(spikes):
            for i in range (0,M):
                curravg+=spikes[len(spikes)-1-i]
            smoothspikes.append(curravg/M)
        else:
            for i in range(window_min, window_max+1):
                curravg+=spikes[i]
            smoothspikes.append(curravg/M)
        window_min +=1
        window_max +=1
    return smoothspikes

In [4]:
raw_data = mne.io.read_raw_edf('Data_chb03/chb03_08.edf').get_data()

time_segment = 60*3 #here we set the lenght of data we gather to track behavior

windows = []
window_length_sec = 5       #here we can set the lenth of the windows
for channel in raw_data:
  windows.append(splitperiod(channel,window_length_sec,256))    #256 samples/s

data = []
for channel in windows:
  data.append(splitperiod(channel,time_segment,(1/window_length_sec)))   #1/sample period(s) = sample frequency(s), 120 sekunder langa segment
  
spike_data = []   #skapa en placeholder lista for all data
for channel in data:
  spike_channel = []  #skapa en placeholder list for en av kanalerna
  for segment in channel:
    spike_segment = []  #skapa en placeholder lista for ett av segmenten
    for window in segment:
      spike_segment.append(spikeamount(window,0.0001,0.04,0.07,256))  #rakna antalet spikes, lagg till det i placeholder segmentet
    spike_segment = smoothspikes(spike_segment,3) #smootha segmentet med M = 3
    spike_channel.append(spike_segment) #segmentet ar klart, lagg till det i kanalen
  spike_data.append(spike_channel)  #kanalen ar klar lagg till den i datan
  
spike_data_mean = []   #skapa en placeholder lista for all data
for channel in data:
  spike_channel = []  #skapa en placeholder list for en av kanalerna
  for segment in channel:
    spike_segment = []  #skapa en placeholder lista for ett av segmenten
    for window in segment:
      spike_segment.append(spikeamount(window,0.0001,0.04,0.07,256))  #rakna antalet spikes, lagg till det i placeholder segmentet
    spike_segment = smoothspikes(spike_segment,3) #smootha segmentet med M = 3 
    spike_channel.append(spike_segment) #segmentet ar klart, lagg till det i kanalen
  spike_data_mean.append(spike_channel)  #kanalen ar klar lagg till den i datan

#nu har vi alltsa en lista som innehaller alla kanaler som listor. kanalerna innehaller 2 minuters segment som listor. segmenten innehaller mangden spikes i varje 2 sekunders fonster som ints.

spike_amounts = []
for channel in spike_data:
  channel_dictionaries = []
  for segment in channel:
    segment_dictionary = {}
    for window in segment:
      try:
        segment_dictionary[window] += 1
      except:
        segment_dictionary[window] = 1
    channel_dictionaries.append(segment_dictionary)
  spike_amounts.append(channel_dictionaries)
i = 0

#used for plotting if pattern wants to be seen
'''
#jag plottar en av kanalerna bara for att det ar kul, varje plot ar ett stort window, plotsen som ar tomma har 0 spikes helt och hallet.
for segment in spike_amounts[0]:
  lists = sorted(segment.items()) # sorted by key, return a list of tuples
  #print(lists)    #Det som printas här är alltså en lista av tupler (mängd spikes, mängd gånger det har hänt) och den är sorterad efter mängd spikes. 
  x, y = zip(*lists) # unpack a list of pairs into two tuples
  plt.plot(x, y)
  print("Seconds passed:" + str(i))
  i += time_segment
  plt.ylim((0,25))
  plt.xlim((0,3))
  plt.show()
'''


sorted_spike_amounts = []
for channel in spike_amounts:
  sorted_spike_amounts_channel = []
  for segment in channel:
    lists = sorted(segment.items())
    sorted_spike_amounts_channel.append(lists)
  sorted_spike_amounts.append(sorted_spike_amounts_channel)


#print(sorted_spike_amounts) #en lista av kanaler. kanalerna ar listor av stora windows. stora windows ar listor av tupler. tuplerna ar pa formen (mangd spikes i ett litet window(i detta fall 5 sek), antal ganger sa manga spikes har skett i detta lilla window). 
#tuplerna ar sorterade i sina stora windows efter det forsta vardet, alltsa mangden spikes.
#som exempel
#[ [ [(0,5),(1,3)],[(0,3),(2,5)], [(0,1), (1,6)] ],[ [(0,3),(1,4)],[(1,2),(3,4)], [(0,6),(1,10)] ] ]
#i denna har vi 2 kanaler med 3 "stora" windows. I kanal 1, stort window 1 ser vi att 0 spikes har skett i 5 av "sma" windows, 1 spike har skett i 3 av dem. I stort window 2 har 0 spikes skett 0 ggr, 2 spikes 5 ggr, och sa vidare
#ett stort window i vart fall ar 2 minuter, litet ar 5 sekunder.


###calculates percentage och derivative ###
#loop through and divide whit sum of number of spikes
#store this value in a list 

percentages = []
for channel in spike_amounts:
    percentages_channel = []
    for window in channel:
        summed = sum(window.values())
        SubSection = []
        for values in window.values():
            SubSection.append(values/summed)
        percentages_channel.append(SubSection)
    percentages.append(percentages_channel)

#creates a list of smooth spike number dvs removes the number of ocurrences from tuples in sorted_spike_amounts
sorted_smooth_spikes = []
for channel in sorted_spike_amounts:
    sorted_smooth_spikes_channel = []
    for window in channel:
        sorted_smooth_spikes_window = []
        for i,values in window:
            sorted_smooth_spikes_window.append(i)
        sorted_smooth_spikes_channel.append(sorted_smooth_spikes_window)
    sorted_smooth_spikes.append(sorted_smooth_spikes_channel) 
          
der = []
for j,values in enumerate(sorted_smooth_spikes):
    der_channel = []
    for i,values1 in enumerate(values):
        if len(diff(values1)):
            temp_der = sum((diff(percentages[j][i])/diff(values1)))/len(diff(values1))
            der_channel.append(temp_der)
    der.append(der_channel)
 
# We now have a list of channels and the mean derivative 

# alarm loop
#need to calculate a treshold
warning = []
treshold_der = 0.05 #5.4444444444444444444444444444444
neg_treshold_der = -0.03
for channel in der:
    warning_channel = []
    for window_der in channel:
        if (window_der) < treshold_der and window_der > neg_treshold_der :
            warning_channel.append(1)
        else:
            warning_channel.append(0)
    warning.append(warning_channel)        
Seizure = []
TimeOfSeizure = 0
for channel in warning:
    Seizure_channel = []
    for i,alarm in enumerate(channel):
        
        if alarm == 1:
            TimeOfSeizure = i*time_segment
            Seizure_channel.append(TimeOfSeizure)
    Seizure.append(Seizure_channel)
print(Seizure)        
#Will print a a list of each channel and at what second it detects a seizure, if it warns two times it will be regarded  
#as a seizure            


Extracting EDF parameters from d:\Epilepsi spike detection\Data_chb03\chb03_08.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\johan\AppData\Local\Temp\ipykernel_24192\3695830947.py:1: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw_data = mne.io.read_raw_edf('Data_chb03/chb03_08.edf').get_data()


[[], [], [], [], [900], [], [], [], [2700], [], [], [], [], [], [], [], [], [], [], [], [], [], []]


In [54]:
# used for analysis of maximum and minimum derivatives of all channels for tuning of above alarm loop.
max_der_per_chan = []
for channel in der:
   if channel != []:
      maxDer =  max((channel))
      max_der_per_chan.append(maxDer)
print(max_der_per_chan)
print(max(max_der_per_chan))


min_der_candidates = []
for channel in der:
   temp_chan = []
   for val in channel:
      if val > 0:   
         temp_chan.append(val)
   min_der_candidates.append(temp_chan)

min_der_per_chan = []
for channel in min_der_candidates:
   if channel != []:
      minDer = min((channel))
   min_der_per_chan.append(minDer)
print(min_der_per_chan)
print(min(min_der_per_chan))

[0.29166666666666663, 0.7083333333333331, 0.05000000000000003, -0.02777777777777768, 2.166666666666667, -0.04166666666666663, -0.611111111111111, -2.5, 0.891891891891892, 1.0416666666666667, -2.0000000000000004, -2.5, 1.3333333333333333, 0.011111111111111094, 0.17499999999999996, 0.13888888888888873, -0.34722222222222227, 0.0999999999999999, 0.06944444444444448, 1.8333333333333335, 0.00396825396825401, 0.17499999999999996]
2.166666666666667
9.25185853854297e-18
